In [ ]:
class FMEXLock {    
    
    // barrier 1 and 2
    volatile int x_fmex = 0;
    volatile int y_fmex = 0;

    // b array -- Java doesn't provide volatile access to array elements
    Integer b_fmex[];
    
    int numthreads;
    
    public void lock(int id)
    {
        while(true)
        {
            b_fmex[id]=1;
            x_fmex=id;
            if (y_fmex!=0) 
            {
                b_fmex[id]=0; 
                while (y_fmex!=0) {}
                continue;
            }
            y_fmex=id;
            if (x_fmex!=id) 
            {
                b_fmex[id] = 0;
                for (int u=0; u<numthreads; u++)
                {
                    while (b_fmex[u]!=0) {}
                }
                if (y_fmex!=id)
                {
                    while(y_fmex!=0) {}
                    continue;    
                }
            }
            return;
        }
    }
    
    public void unlock(int id)
    {        
        y_fmex=0;
        b_fmex[id]=0;
    }
    
    FMEXLock (int nt)
    {
        numthreads = nt;
        b_fmex = new Integer[numthreads];
    }
}

class FMEXWorks implements Runnable
{
  int thread_id;
  FMEXLock lock;
  
  static int sharedvar = 0;

  public void run ()
  { 
    for ( int i=0; i<10000; i++ )
    {
        lock.lock(thread_id);
        sharedvar++;
        lock.unlock(thread_id);      
    }
  }
  
  FMEXWorks(int tid, FMEXLock lk)
  {
    thread_id = tid;
    lock = lk;
  }
}

int numthreads = 4;
FMEXLock lock = new FMEXLock(4);
Thread[] threads = new Thread[numthreads];

// create and start thread objects
for ( int i=0; i<numthreads; i++ )
{
    threads[i] = new Thread ( new FMEXWorks(i, lock) );
    threads[i].start();
}

// Await the completion of all threads
for ( int i=0; i<numthreads; i++ )
{
    threads[i].join();
}

System.out.println("Shared variable = " + FMEXWorks.sharedvar);